In [1]:
import pickle
import numpy as np
import pandas as pd

from collections import Counter

from sklearn.metrics import f1_score, make_scorer, confusion_matrix, \
    classification_report
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score, \
    StratifiedShuffleSplit, RandomizedSearchCV

# Load learn data

In [2]:
with open("../Data/Learn/labels.pkl", "rb") as f:
    learn_labels = pickle.load(f)

with open("../Data/Learn/sentences.pkl", "rb") as f:
    learn_sentences = pickle.load(f)

with open("../Data/Learn/sequences.pkl", "rb") as f:
    learn_sequences = pickle.load(f)

with open("../Data/dict.pkl", "rb") as f:
    vocabulary = pickle.load(f)
    reverse_vocabulary = {v: k for k, v in vocabulary.items()}

# Data exploration

In [3]:
def decode_sequence(sequence, decoding_dict):
    return " ".join((decoding_dict[x] for x in sequence))

In [4]:
print(len(vocabulary), len(reverse_vocabulary))
print(min(reverse_vocabulary.keys()), max(reverse_vocabulary.keys()))
sorted(vocabulary.keys(), key=len, reverse=True)[:5]

30432 30432
0 30431


['déresponsabiliseraient',
 'interprofessionnelles',
 'interprofessionnalité',
 'socioprofessionnelles',
 'intergouvernementale']

In [5]:
Counter(learn_labels)

Counter({'C': 39912, 'M': 6018})

In [6]:
print(len(learn_sentences))
learn_sentences[:5]

45930


["J'aurai l'occasion de dire aux Français comment notre enseignement devra évoluer pour permettre à chaque jeune de trouver sa place, d'entrer dans le monde du travail, de savoir s'adapter et, à partir de là, d'acquérir, tout au long de la vie, de nouvelles compétences et de nouveaux savoirs.",
 'Il est nécessaire.',
 "Dans votre coeur et dans votre vie, la confiance et l'enthousiasme l'emportent sur le doute.",
 "Pour conduire ce débat dans un esprit de véritable dialogue, je compte, si nos partenaires en sont d'accord, inviter au prochain sommet du G7, à Lyon, pour une séance de travail, le secrétaire général des Nations Unies, le président de la Banque mondiale et le directeur général du Fonds monétaire international.",
 "La France et l'Europe construiront ainsi un avenir de coopération avec un Proche-Orient pacifié, stable, prospère, libéré des menaces de la guerre, de la prolifération, du terrorisme."]

In [7]:
print(len(learn_sequences))
[decode_sequence(x, reverse_vocabulary) for x in learn_sequences][:5]

45930


["j' aurai l' occasion de dire aux français comment notre enseignement devra évoluer pour permettre à chaque jeune de trouver sa place , d' entrer dans le monde du travail , de savoir s' adapter et , à partir de là , d' acquérir , tout au long de la vie , de nouvelles compétences et de nouveaux savoirs .",
 'il est nécessaire .',
 "dans votre coeur et dans votre vie , la confiance et l' enthousiasme l' emportent sur le doute .",
 "pour conduire ce débat dans un esprit de véritable dialogue , je compte , si nos partenaires en sont d' accord , inviter au prochain sommet du g7 , à lyon , pour une séance de travail , le secrétaire général des nations unies , le président de la banque mondiale et le directeur général du fonds monétaire international .",
 "la france et l' europe construiront ainsi un avenir de coopération avec un proche - orient pacifié , stable , prospère , libéré des menaces de la guerre , de la prolifération , du terrorisme ."]

# Data splitting

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    learn_sequences, learn_labels, test_size=0.3,
    shuffle=True, stratify=learn_labels, random_state=42
)
print(len(X_train), len(y_train), len(X_test), len(y_test))
Counter(y_train), Counter(y_test)

32151 32151 13779 13779


(Counter({'C': 27938, 'M': 4213}), Counter({'C': 11974, 'M': 1805}))

# Utils

In [16]:
def f1(y_true, y_pred):
    return f1_score(y_true, y_pred, average="macro")

scoring = make_scorer(f1)

def score_model(model):
    model.fit(X_train, y_train)
    return scoring(model, X_test, y_test)


def report_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))


def grid_search(model, param_grid, n_iter):
    rs = RandomizedSearchCV(
        model, param_grid, 
        n_iter=n_iter, 
        scoring=scoring,
        cv=StratifiedShuffleSplit(10, test_size=0.3, random_state=42),
        n_jobs=-1,
        random_state=42,
    )
    rs.fit(X_train, y_train)
    return pd.DataFrame(rs.cv_results_).sort_values(
        "mean_test_score", ascending=False)


def tokens_to_str(tokens):
    return list(map(str, tokens))

# Baseline models

### Majority

In [10]:
class Model(BaseEstimator, ClassifierMixin):
    def fit(self, X, y):
        return self
    def predict(self, X):
        return np.full(len(X), 'C')

In [11]:
"Test score: %s" % score_model(Model())

/home/omar/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'Test score: 0.46495553916048615'

### N-grams + Naive Bayes

In [17]:
Model = lambda: Pipeline([
    ("vectorizer", CountVectorizer(lowercase=False, tokenizer=tokens_to_str)),
    ("classifier", MultinomialNB()),
])

In [ ]:
results = grid_search(Model(), {
    "vectorizer__ngram_range": [(1, 1), (1, 2), (2, 2), (1, 3), (2, 3), (3, 3)],
    "classifier__alpha": np.logspace(-3, 0, 10),
}, n_iter=60)

In [107]:
best_params = results.params.iloc[0]
print(best_params)
"Test score: %s" % score_model(Model().set_params(**best_params))

{'vectorizer__ngram_range': (1, 2), 'classifier__alpha': 0.1}


'Test score: 0.7752731066507919'

In [51]:
print(best_params)
report_model(Model().set_params(**best_params))

{'vectorizer__ngram_range': (1, 2), 'classifier__alpha': 0.1}
[[11277   697]
 [  709  1096]]
              precision    recall  f1-score   support

           C      0.941     0.942     0.941     11974
           M      0.611     0.607     0.609      1805

   micro avg      0.898     0.898     0.898     13779
   macro avg      0.776     0.774     0.775     13779
weighted avg      0.898     0.898     0.898     13779

